In [5]:
import torch
import torch.nn.functional as F
torch.set_printoptions(profile='full')

In [17]:
proto = torch.Tensor([[1,2,3]])
print(proto.size())
print(proto.squeeze(0).shape)
Z_p = torch.Tensor([[4,5,6], [7,8,9]])
Z_c = torch.Tensor([[10,11,12], [13,14,15]])
delta = Z_c - Z_p
print(delta)
dist = torch.sqrt(torch.sum(torch.pow(Z_p - proto, 2), dim=1))
print(dist)
dist_mean = torch.mean(dist)
print(dist_mean)
dist_variance = torch.mean(torch.pow(dist - dist_mean, 2))
print(dist_variance)
w = torch.exp(-torch.pow(dist, 2) / (2 * dist_variance))
print(w)
W = torch.sum(w)
print(W)
torch.sum(torch.unsqueeze(w, 1) * delta, dim=0) / W

torch.Size([1, 3])
torch.Size([3])
tensor([[6., 6., 6.],
        [6., 6., 6.]])
tensor([ 5.1962, 10.3923])
tensor(7.7942)
tensor(6.7500)
tensor([0.1353, 0.0003])
tensor(0.1357)


tensor([6., 6., 6.])

In [2]:
embedding = torch.randn(5)
F.normalize(embedding, dim=0)

tensor([-0.3480, -0.7114,  0.0283,  0.5568,  0.2489])

In [6]:
batch = F.normalize(torch.randn(10,5), dim=1)
batch

tensor([[-6.4204e-03,  6.4207e-01,  1.4702e-01, -1.3592e-02,  7.5227e-01],
        [ 3.0482e-01, -5.1897e-01, -7.9719e-01,  1.8418e-02, -4.3599e-02],
        [-4.8780e-02,  2.9550e-01, -2.3958e-01, -9.2353e-01, -4.6353e-05],
        [-2.8724e-02,  2.1884e-01,  8.4716e-01,  3.7832e-01,  3.0080e-01],
        [ 2.5956e-01,  6.6577e-01, -4.3384e-01,  4.8069e-01,  2.6475e-01],
        [ 5.1806e-01, -2.4194e-01, -6.9662e-01, -4.6739e-02,  4.3084e-01],
        [-8.1449e-01, -3.7669e-01, -8.1611e-02, -3.4818e-01, -2.5850e-01],
        [-6.0941e-01,  2.6994e-01,  4.9418e-01, -5.5813e-01, -4.9405e-03],
        [ 1.0311e-01, -6.7575e-01, -5.0741e-01, -4.7478e-02, -5.2250e-01],
        [-9.3107e-02, -2.9841e-01, -8.5853e-01,  3.9782e-01,  8.3371e-02]])

In [7]:
labels = torch.Tensor([0,1,1,0,0,2,2,0,3,3])

In [8]:
temperature = 0.01

In [9]:
dot_product_tempered = torch.mm(batch, batch.T) / temperature  # z_i \dot z_j / \tau
dot_product_tempered.size()  # [dim, dim](10,10)
dot_product_tempered

tensor([[100.0000, -48.5429,  16.7340,  48.6387,  55.4649,   6.3653, -43.8359,
          25.3762, -90.1557, -25.9919],
        [-48.5429, 100.0000,   0.5757, -80.3821,   7.6767,  81.9165,   1.7140,
         -72.9874,  80.8538,  81.4591],
        [ 16.7340,   0.5757, 100.0000, -48.6294, -15.5924,  11.3274,  26.9539,
          50.6548,  -3.9278, -24.5351],
        [ 48.6387, -80.3821, -48.6294, 100.0000,   3.2195, -54.6056, -33.7658,
          28.2592, -75.5831, -61.4363],
        [ 55.4649,   7.6767, -15.5924,   3.2195, 100.0000,  36.7210, -66.2602,
         -46.2450, -36.4151,  36.2922],
        [  6.3653,  81.9165,  11.3274, -54.6056,  36.7210, 100.0000, -36.9058,
         -70.1317,  34.7486,  63.9355],
        [-43.8359,   1.7140,  26.9539, -33.7658, -66.2602, -36.9058, 100.0000,
          54.9950,  36.3576,   9.8246],
        [ 25.3762, -72.9874,  50.6548,  28.2592, -46.2450, -70.1317,  54.9950,
         100.0000, -46.6925, -67.0527],
        [-90.1557,  80.8538,  -3.9278, -75.5831,

In [10]:
max_ele = torch.max(dot_product_tempered, dim=1, keepdim=True)
print(max_ele)
dot_product_tempered = dot_product_tempered - max_ele[0]
print(dot_product_tempered)
exp_dot_tempered = (
    torch.exp(dot_product_tempered) 
    + 1e-5
)  # exp(z_i \dot z_j / \tau)，每一行代表和每个其他嵌入的点积
exp_dot_tempered

torch.return_types.max(
values=tensor([[100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000]]),
indices=tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]]))
tensor([[   0.0000, -148.5429,  -83.2660,  -51.3613,  -44.5351,  -93.6347,
         -143.8359,  -74.6238, -190.1557, -125.9919],
        [-148.5429,    0.0000,  -99.4243, -180.3821,  -92.3233,  -18.0835,
          -98.2860, -172.9874,  -19.1462,  -18.5409],
        [ -83.2660,  -99.4243,    0.0000, -148.6294, -115.5924,  -88.6726,
          -73.0461,  -49.3452, -103.9278, -124.5350],
        [ -51.3613, -180.3821, -148.6294,    0.0000,  -96.7805, -154.6056,
         -133.7658,  -71.7408, -175.5831, -161.4363],
        [ -44.5351,  -92.3233, -115.5924,  -96.7805,    0.0000,  -63.2790,
         -166.2602, -146.2450, -136.

tensor([[1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0014e-05,
         1.0000e-05, 1.0000e-05, 1.0005e-05, 1.0009e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0014e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000

In [11]:
# 分母是z_i与batch中其他嵌入的点积，因此要屏蔽掉exp_dot_tempered中自己与自己相乘的部分，即对角线上的取值
unsqeezed_labels = labels.unsqueeze(1)  # [[0],[1],[1],[0],[0],[2],[2],[0],[3],[3]]
repeated_labels = unsqeezed_labels.repeat(1, labels.shape[0])  # 每一行都是batch_size长的全为对应的样本的标签的向量
mask_similar_class = (repeated_labels == labels)
print(mask_similar_class)
mask_anchor_out = (1 - torch.eye(exp_dot_tempered.shape[0]))  # 对角线为0其他全为1的batch_size * batch_size的张量
print(mask_anchor_out)

tensor([[ True, False, False,  True,  True, False, False,  True, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True]])
tensor([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],

In [17]:
mask_combined = mask_similar_class * mask_anchor_out
mask_combined # 每个样本在batch中相同样本的其他样本对应的位置为1，不同的为0


tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [19]:
cardinality_per_samples = torch.sum(mask_combined, dim=1)
cardinality_per_samples  # batch中每个样本的与他标签相同的其他样本的数量

tensor([3., 1., 1., 3., 3., 1., 1., 3., 1., 1.])

In [46]:
fenmu = torch.sum(exp_dot_tempered * mask_anchor_out, dim=1, keepdim=True)
print(fenmu)
log_prob = -torch.log(exp_dot_tempered / fenmu)
print(log_prob)

tensor([[9.0289e-05],
        [1.3500e-04],
        [1.3500e-04],
        [9.0001e-05],
        [9.0289e-05],
        [9.0000e-05],
        [9.0001e-05],
        [9.0000e-05],
        [9.0000e-05],
        [9.0000e-05]])
tensor([[-9.3125,  2.2004,  2.2004,  2.2004,  2.1720,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.6027, -8.9102,  0.8979,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.6027,  0.8979, -8.9102,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1720,  2.2004,  2.2004,  2.2004, -9.3125,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,
          2.1972,  2.1972],
        [ 2.1972

In [47]:
log = log_prob * mask_combined
print(log)
supervised_contrastive_loss_per_sample = torch.sum(log, dim=1) / cardinality_per_samples
print(supervised_contrastive_loss_per_sample)
supervised_contrastive_loss = torch.sum(supervised_contrastive_loss_per_sample)
print(supervised_contrastive_loss)


tensor([[-0.0000, 0.0000, 0.0000, 2.2004, 2.1720, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, -0.0000, 0.8979, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.8979, -0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000, 2.1972, 0.0000,
         0.0000],
        [2.1720, 0.0000, 0.0000, 2.2004, -0.0000, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972, -0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, 2.1972, 2.1972, 0.0000, 0.0000, -0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000,
         2.1972],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972,
         -0.0000]])